In [1]:
!pip install darts --quiet
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime, timedelta
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
import darts
darts.__version__

'0.26.0'

In [3]:
import warnings
warnings.filterwarnings('ignore')

### Train

In [4]:
df_train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
display(df_train.head())

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [5]:
df_train.iloc[np.where(df_train['date'].str.contains('12-25'))]

,id,date,store_nbr,family,sales,onpromotion


### holidays events

In [6]:
df_holidays_events = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
display(df_holidays_events.head(7))

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False
6,2012-06-23,Holiday,Local,Guaranda,Cantonizacion de Guaranda,False


In [7]:
df_holidays_events.groupby(['type', 'locale'])['locale_name'].count()

type        locale  
Additional  Local        11
            National     40
Bridge      National      5
Event       National     56
Holiday     Local       137
            National     60
            Regional     24
Transfer    Local         4
            National      8
Work Day    National      5
Name: locale_name, dtype: int64

### oil

In [8]:
df_oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
display(df_oil.head())

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


### stores

In [9]:
df_stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
display(df_stores.head())

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


### transactions

In [10]:
df_transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
display(df_transactions.head())

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


### test data

In [11]:
df_test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
display(df_test.head())

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


## preprocessing

In [12]:
family_list = df_train['family'].unique()
store_list = df_stores['store_nbr'].unique()
display(family_list)
display(store_list)

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54])

In [13]:
train_merged = pd.merge(df_train, df_stores, on ='store_nbr')
train_merged = train_merged.sort_values(["store_nbr","family","date"])
train_merged = train_merged.astype({"store_nbr":'str', "family":'str', "city":'str',
                          "state":'str', "type":'str', "cluster":'str'})

train_merged

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
33,1782,2013-01-02,1,AUTOMOTIVE,2.0,0,Quito,Pichincha,D,13
66,3564,2013-01-03,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13
99,5346,2013-01-04,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13
132,7128,2013-01-05,1,AUTOMOTIVE,5.0,0,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...,...
2778467,2993627,2017-08-11,54,SEAFOOD,0.0,0,El Carmen,Manabi,C,3
2778500,2995409,2017-08-12,54,SEAFOOD,1.0,1,El Carmen,Manabi,C,3
2778533,2997191,2017-08-13,54,SEAFOOD,2.0,0,El Carmen,Manabi,C,3
2778566,2998973,2017-08-14,54,SEAFOOD,0.0,0,El Carmen,Manabi,C,3


In [14]:
train_merged.dtypes

id               int64
date            object
store_nbr       object
family          object
sales          float64
onpromotion      int64
city            object
state           object
type            object
cluster         object
dtype: object

## TimeSeries

In [15]:
from darts import TimeSeries
from tqdm import tqdm

In [16]:
family_TS_dict = {}

for family in tqdm(family_list):
    df_family = train_merged.loc[train_merged['family'] == family]

    list_of_TS_family = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col="date",
                                group_cols=["store_nbr","family"], # columns for grouping time series
                                static_cols=["city","state","type","cluster"], # static covariates
                                value_cols="sales", # target
                                fill_missing_dates=True, # filling missing dates, remember Dec 25th
                                freq='D' # days
                                )
    for ts in list_of_TS_family:
            ts = ts.astype(np.float32)

            
    list_of_TS_family = sorted(list_of_TS_family, key=lambda ts: int(ts.static_covariates_values()[0,0]))
    
    family_TS_dict[family] = list_of_TS_family

100%|██████████| 33/33 [00:28<00:00,  1.17it/s]


In [17]:
family_TS_dict.keys()

dict_keys(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS', 'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS', 'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE', 'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES', 'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE', 'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE', 'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY', 'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES', 'SEAFOOD'])

In [18]:
family_TS_dict['AUTOMOTIVE'][0]

<TimeSeries (DataArray) (date: 1688, component: 1, sample: 1)>
array([[[0.]],

       [[2.]],

       [[3.]],

       ...,

       [[1.]],

       [[1.]],

       [[4.]]])
Coordinates:
  * date       (date) datetime64[ns] 2013-01-01 2013-01-02 ... 2017-08-15
  * component  (component) object 'sales'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates store_nbr      family   city      s...
    hierarchy:          None

In [19]:
family_TS_dict['AUTOMOTIVE'][0].static_covariates

static_covariates,store_nbr,family,city,state,type,cluster
component,,,,,,
sales,1,AUTOMOTIVE,Quito,Pichincha,D,13


### Normalizing time series

In [20]:
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer, MissingValuesFiller, InvertibleMapper
from sklearn.preprocessing import OrdinalEncoder

In [21]:
family_pipeline_dict = {}
family_TS_transformed_dict = {}

for key in tqdm(family_TS_dict):
    train_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
    static_cov_transformer = StaticCovariatesTransformer(verbose=False, transformer_cat = OrdinalEncoder(), name="Encoder")
    log_transformer = InvertibleMapper(np.log1p, np.expm1, verbose=False, n_jobs=-1, name="Log-Transform")   
    train_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

    train_pipeline = Pipeline([train_filler,
                             static_cov_transformer,
                             log_transformer,
                             train_scaler])

    training_transformed = train_pipeline.fit_transform(family_TS_dict[key])
    family_pipeline_dict[key] = train_pipeline
    family_TS_transformed_dict[key] = training_transformed

100%|██████████| 33/33 [00:43<00:00,  1.33s/it]


In [22]:
family_TS_transformed_dict['AUTOMOTIVE'][0]

<TimeSeries (DataArray) (date: 1688, component: 1, sample: 1)>
array([[[0.        ]],

       [[0.36672579]],

       [[0.46275643]],

       ...,

       [[0.23137821]],

       [[0.23137821]],

       [[0.53724357]]])
Coordinates:
  * date       (date) datetime64[ns] 2013-01-01 2013-01-02 ... 2017-08-15
  * component  (component) object 'sales'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr  family  city  state  ty...
    hierarchy:          None

## Date

In [23]:
from darts.utils.timeseries_generation import datetime_attribute_timeseries

In [24]:
full_time_period = pd.date_range(start='2013-01-01', end='2017-08-31', freq='D')

year = datetime_attribute_timeseries(time_index = full_time_period, attribute='year')
month = datetime_attribute_timeseries(time_index= full_time_period, attribute='month')
day = datetime_attribute_timeseries(time_index = full_time_period, attribute='day')
dayofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute = 'dayofyear')
weekday = datetime_attribute_timeseries(time_index = full_time_period , attribute='dayofweek')
weekofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute='weekofyear')
timesteps = TimeSeries.from_times_and_values(times = full_time_period,
                                            values = np.arange(len(full_time_period)),
                                            columns= ['linear_increase'])

time_cov = year.stack(month).stack(day).stack(dayofyear).stack(weekday).stack(weekofyear).stack(timesteps)
time_cov = time_cov.astype(np.float32)

In [25]:
display(print(time_cov.components.values))
display(time_cov[100])

['year' 'month' 'day' 'dayofyear' 'dayofweek' 'weekofyear'
 'linear_increase']


None

<TimeSeries (DataArray) (time: 1, component: 7, sample: 1)>
array([[[2013.],
        [   4.],
        [  11.],
        [ 101.],
        [   3.],
        [  15.],
        [ 100.]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2013-04-11
  * component  (component) object 'year' 'month' ... 'linear_increase'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [26]:
time_cov_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
time_cov_train, time_cov_val = time_cov.split_before(pd.Timestamp('20170816'))
time_cov_scaler.fit(time_cov_train)
time_cov_transformed = time_cov_scaler.transform(time_cov)

### OIL DF 

In [27]:
from darts.models.filtering.moving_average_filter import MovingAverageFilter

In [28]:
# Oil Price
oil = TimeSeries.from_dataframe(df_oil, 
                                time_col = 'date', 
                                value_cols = ['dcoilwtico'],
                                freq = 'D')

oil = oil.astype(np.float32)

# Transform
oil_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
oil_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
oil_pipeline = Pipeline([oil_filler, oil_scaler])
oil_transformed = oil_pipeline.fit_transform(oil)

# Moving Averages for Oil Price
oil_moving_average_7 = MovingAverageFilter(window=7)
oil_moving_average_28 = MovingAverageFilter(window=28)

oil_moving_averages = []

ma_7 = oil_moving_average_7.filter(oil_transformed).astype(np.float32)
ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new='oil_ma_7')
ma_28 = oil_moving_average_28.filter(oil_transformed).astype(np.float32)
ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new='oil_ma_28')
oil_moving_averages = ma_7.stack(ma_28)

In [29]:
oil_moving_averages

<TimeSeries (DataArray) (date: 1704, component: 2, sample: 1)>
array([[[0.79240197],
        [0.796154  ]],

       [[0.7925303 ],
        [0.7960361 ]],

       [[0.79266846],
        [0.796688  ]],

       ...,

       [[0.24278495],
        [0.25032222]],

       [[0.24149394],
        [0.25014064]],

       [[0.24079117],
        [0.25054285]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 2013-01-01 2013-01-02 ... 2017-08-31
  * component  (component) object 'oil_ma_7' 'oil_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

### Holidays DF

In [30]:
def holiday_list(df_stores):

    listofseries = []
    
    for i in range(0,len(df_stores)):        
            df_holiday_dummies = pd.DataFrame(columns=['date'])
            df_holiday_dummies["date"] = df_holidays_events["date"]
    
            df_holiday_dummies["national_holiday"] = np.where(((df_holidays_events["type"] == "Holiday") & (df_holidays_events["locale"] == "National")), 1, 0)

            df_holiday_dummies["earthquake_relief"] = np.where(df_holidays_events['description'].str.contains('Terremoto Manabi'), 1, 0)

            df_holiday_dummies["christmas"] = np.where(df_holidays_events['description'].str.contains('Navidad'), 1, 0)

            df_holiday_dummies["football_event"] = np.where(df_holidays_events['description'].str.contains('futbol'), 1, 0)

            df_holiday_dummies["national_event"] = np.where(((df_holidays_events["type"] == "Event") & (df_holidays_events["locale"] == "National") & (~df_holidays_events['description'].str.contains('Terremoto Manabi')) & (~df_holidays_events['description'].str.contains('futbol'))), 1, 0)

            df_holiday_dummies["work_day"] = np.where((df_holidays_events["type"] == "Work Day"), 1, 0)

            df_holiday_dummies["local_holiday"] = np.where(((df_holidays_events["type"] == "Holiday") & ((df_holidays_events["locale_name"] == df_stores['state'][i]) | (df_holidays_events["locale_name"] == df_stores['city'][i]))), 1, 0)
                     
            listofseries.append(df_holiday_dummies)

    return listofseries

In [31]:
def remove_0_and_duplicates(holiday_list):

    listofseries = []
    
    for i in range(0,len(holiday_list)):         
            df_holiday_per_store = list_of_holidays_per_store[i].set_index('date')

            df_holiday_per_store = df_holiday_per_store.loc[~(df_holiday_per_store==0).all(axis=1)]
            
            df_holiday_per_store = df_holiday_per_store.groupby('date').agg({'national_holiday':'max', 'earthquake_relief':'max', 
                                   'christmas':'max', 'football_event':'max', 
                                   'national_event':'max', 'work_day':'max', 
                                   'local_holiday':'max'}).reset_index()

            listofseries.append(df_holiday_per_store)

    return listofseries

In [32]:
def holiday_TS_list_54(holiday_list):
    listofseries = []
    
    for i in range(0,54):
            holidays_TS = TimeSeries.from_dataframe(list_of_holidays_per_store[i], 
                                        time_col = 'date',
                                        fill_missing_dates=True,
                                        fillna_value=0,
                                        freq='D')
            
            holidays_TS = holidays_TS.slice(pd.Timestamp('20130101'),pd.Timestamp('20170831'))
            holidays_TS = holidays_TS.astype(np.float32)
            listofseries.append(holidays_TS)

    return listofseries

In [33]:
list_of_holidays_per_store = holiday_list(df_stores)
list_of_holidays_per_store = remove_0_and_duplicates(list_of_holidays_per_store)
list_of_holidays_store = holiday_TS_list_54(list_of_holidays_per_store)

holidays_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
holidays_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

holidays_pipeline = Pipeline([holidays_filler, holidays_scaler])
holidays_transformed = holidays_pipeline.fit_transform(list_of_holidays_store)

In [34]:
display(len(holidays_transformed))
display(holidays_transformed[0].components.values)
display(holidays_transformed[0][0])

54

array(['national_holiday', 'earthquake_relief', 'christmas',
       'football_event', 'national_event', 'work_day', 'local_holiday'],
      dtype=object)

<TimeSeries (DataArray) (date: 1, component: 7, sample: 1)>
array([[[1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 2013-01-01
  * component  (component) object 'national_holiday' ... 'local_holiday'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

### OnPromotion

In [35]:
df_promotion = pd.concat([df_train, df_test], axis=0)
df_promotion = df_promotion.sort_values(["store_nbr","family","date"])
display(df_promotion.tail())

family_promotion_dict = {}

for family in tqdm(family_list):
    df_family = df_promotion.loc[df_promotion['family'] == family]

    list_of_TS_promo = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col="date",
                                group_cols=["store_nbr","family"],
                                value_cols="onpromotion",
                                fill_missing_dates=True,
                                freq='D')

    for ts in list_of_TS_promo:
        ts = ts.astype(np.float32)

    family_promotion_dict[family] = list_of_TS_promo

,id,date,store_nbr,family,sales,onpromotion
21251,3022139,2017-08-27,54,SEAFOOD,NaN,0
23033,3023921,2017-08-28,54,SEAFOOD,NaN,0
24815,3025703,2017-08-29,54,SEAFOOD,NaN,0
26597,3027485,2017-08-30,54,SEAFOOD,NaN,0
28379,3029267,2017-08-31,54,SEAFOOD,NaN,0


100%|██████████| 33/33 [00:29<00:00,  1.13it/s]


In [36]:
family_promotion_dict['SEAFOOD'][0]

<TimeSeries (DataArray) (date: 1704, component: 1, sample: 1)>
array([[[0.]],

       [[0.]],

       [[0.]],

       ...,

       [[0.]],

       [[1.]],

       [[0.]]])
Coordinates:
  * date       (date) datetime64[ns] 2013-01-01 2013-01-02 ... 2017-08-31
  * component  (component) object 'onpromotion'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr   family\ncomponent     ...
    hierarchy:          None

In [37]:
promotion_transformed_dict = {}

for key in tqdm(family_promotion_dict):
    promo_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
    promo_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

    promo_pipeline = Pipeline([promo_filler,
                             promo_scaler])

    promotion_transformed = promo_pipeline.fit_transform(family_promotion_dict[key])

    # Moving Averages for Promotion Family Dictionaries
    promo_moving_average_7 = MovingAverageFilter(window=7)
    promo_moving_average_28 = MovingAverageFilter(window=28)

    promotion_covs = []

    for ts in promotion_transformed:
        ma_7 = promo_moving_average_7.filter(ts)
        ma_7 = TimeSeries.from_series(ma_7.pd_series())  
        ma_7 = ma_7.astype(np.float32)
        ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="promotion_ma_7")
        ma_28 = promo_moving_average_28.filter(ts)
        ma_28 = TimeSeries.from_series(ma_28.pd_series())  
        ma_28 = ma_28.astype(np.float32)
        ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="promotion_ma_28")
        promo_and_mas = ts.stack(ma_7).stack(ma_28)
        promotion_covs.append(promo_and_mas)

    promotion_transformed_dict[key] = promotion_covs

100%|██████████| 33/33 [01:15<00:00,  2.30s/it]


In [38]:
display(promotion_transformed_dict['SEAFOOD'][0].components)
promotion_transformed_dict['SEAFOOD'][0]

Index(['onpromotion', 'promotion_ma_7', 'promotion_ma_28'], dtype='object', name='component')

<TimeSeries (DataArray) (date: 1704, component: 3, sample: 1)>
array([[[0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ]],

       ...,

       [[0.        ],
        [0.07142857],
        [0.13445379]],

       [[0.14285714],
        [0.05714286],
        [0.14285715]],

       [[0.        ],
        [0.07142857],
        [0.15238096]]])
Coordinates:
  * date       (date) datetime64[ns] 2013-01-01 2013-01-02 ... 2017-08-31
  * component  (component) object 'onpromotion' ... 'promotion_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr   family\nglobal_compone...
    hierarchy:          None

### Group the covariates

In [39]:
general_covariates = time_cov_transformed.stack(oil_transformed).stack(oil_moving_averages)

In [40]:
general_covariates[0]

<TimeSeries (DataArray) (time: 1, component: 10, sample: 1)>
array([[[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.16666667],
        [0.        ],
        [0.        ],
        [0.7929646 ],
        [0.79240197],
        [0.796154  ]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2013-01-01
  * component  (component) object 'year' 'month' ... 'oil_ma_7' 'oil_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [41]:
store_covariates_future = []

for store in range(0,len(store_list)):
    stacked_covariates = holidays_transformed[store].stack(general_covariates)  
    store_covariates_future.append(stacked_covariates)

In [42]:
store_covariates_future[0][0]

<TimeSeries (DataArray) (date: 1, component: 17, sample: 1)>
array([[[1.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.16666667],
        [0.        ],
        [0.        ],
        [0.7929646 ],
        [0.79240197],
        [0.796154  ]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 2013-01-01
  * component  (component) object 'national_holiday' ... 'oil_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [43]:
future_covariates_dict = {}

for key in tqdm(promotion_transformed_dict):
    promotion_family = promotion_transformed_dict[key]
    covariates_future = [promotion_family[i].stack(store_covariates_future[i]) for i in range(0,len(promotion_family))]
    future_covariates_dict[key] = covariates_future

100%|██████████| 33/33 [00:05<00:00,  6.04it/s]


In [44]:
future_covariates_dict['SEAFOOD'][0].components

Index(['onpromotion', 'promotion_ma_7', 'promotion_ma_28', 'national_holiday',
       'earthquake_relief', 'christmas', 'football_event', 'national_event',
       'work_day', 'local_holiday', 'year', 'month', 'day', 'dayofyear',
       'dayofweek', 'weekofyear', 'linear_increase', 'dcoilwtico', 'oil_ma_7',
       'oil_ma_28'],
      dtype='object', name='component')

### Past Covariates - Transactions

In [45]:
df_transactions.sort_values(['store_nbr','date'], inplace= True)

TS_transactions_list = TimeSeries.from_group_dataframe(
                                df_transactions,
                                time_col="date",
                                group_cols=["store_nbr"],
                                value_cols="transactions",
                                fill_missing_dates=True,
                                freq='D')

transactions_list = []

for ts in TS_transactions_list:
            series = TimeSeries.from_series(ts.pd_series())
            series = series.astype(np.float32)
            transactions_list.append(series)
            
# as the transactions dataframe have for store_nbr 24 transactions from 01-01-2013 but every store_nbr have data from 02-01-2013
transactions_list[24] = transactions_list[24].slice(start_ts=pd.Timestamp('20130102'), end_ts=pd.Timestamp('20170815'))

transactions_list_full = []

for ts in transactions_list:
    if ts.start_time() > pd.Timestamp('20130101'):
        end_time = (ts.start_time() - timedelta(days=1))
        delta = end_time - pd.Timestamp('20130101')
        zero_series = TimeSeries.from_times_and_values(
                                  times=pd.date_range(start=pd.Timestamp('20130101'), 
                                  end=end_time, freq="D"),
                                  values=np.zeros(delta.days+1))
        ts = zero_series.append(ts)
        ts = ts.with_columns_renamed(col_names=ts.components, col_names_new="transactions")
        transactions_list_full.append(ts)

transactions_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
transactions_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

transactions_pipeline = Pipeline([transactions_filler, transactions_scaler])
transactions_transformed = transactions_pipeline.fit_transform(transactions_list_full)

In [46]:
transactions_transformed[0]

<TimeSeries (DataArray) (time: 1688, component: 1, sample: 1)>
array([[[0.        ]],

       [[0.69831293]],

       [[0.60635131]],

       ...,

       [[0.13761164]],

       [[0.57327158]],

       [[0.5600397 ]]])
Coordinates:
  * time       (time) datetime64[ns] 2013-01-01 2013-01-02 ... 2017-08-15
  * component  (component) object 'transactions'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

### IDS FOR REFERENCE


In [47]:
df_indexes = pd.concat([df_train, df_test])
df_indexes = df_indexes.drop(['onpromotion'], axis=1)
df_indexes = df_indexes.sort_values(by=['store_nbr', 'family'])
df_indexes.date = pd.to_datetime(df_indexes.date)
df_indexes

,id,date,store_nbr,family,sales
0,0,2013-01-01,1,AUTOMOTIVE,0.0
1782,1782,2013-01-02,1,AUTOMOTIVE,2.0
3564,3564,2013-01-03,1,AUTOMOTIVE,3.0
5346,5346,2013-01-04,1,AUTOMOTIVE,3.0
7128,7128,2013-01-05,1,AUTOMOTIVE,5.0
...,...,...,...,...,...
21251,3022139,2017-08-27,54,SEAFOOD,NaN
23033,3023921,2017-08-28,54,SEAFOOD,NaN
24815,3025703,2017-08-29,54,SEAFOOD,NaN
26597,3027485,2017-08-30,54,SEAFOOD,NaN


In [48]:
df_indexes = df_indexes.set_index('date')
display(df_indexes)

,id,store_nbr,family,sales
date,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0
2013-01-02,1782,1,AUTOMOTIVE,2.0
2013-01-03,3564,1,AUTOMOTIVE,3.0
2013-01-04,5346,1,AUTOMOTIVE,3.0
2013-01-05,7128,1,AUTOMOTIVE,5.0
...,...,...,...,...
2017-08-27,3022139,54,SEAFOOD,NaN
2017-08-28,3023921,54,SEAFOOD,NaN
2017-08-29,3025703,54,SEAFOOD,NaN


In [49]:
date_range = pd.date_range(start=df_indexes.index.min(), end=df_indexes.index.max(), freq='D')
df_indexes_filled = pd.DataFrame(columns=df_indexes.columns)

for family in tqdm(family_list):
    for store in store_list:
        temp_df = df_indexes.iloc[np.where((df_indexes.family == family) & (df_indexes.store_nbr == store))]
        temp_df = temp_df.reindex(date_range).fillna({'id': np.nan, 'store_nbr': store, 'family': family, 'sales': np.nan})
        df_indexes_filled = pd.concat([df_indexes, temp_df])
        
df_indexes_filled

100%|██████████| 33/33 [08:34<00:00, 15.60s/it]


,id,store_nbr,family,sales
2013-01-01,0.0,1.0,AUTOMOTIVE,0.0
2013-01-02,1782.0,1.0,AUTOMOTIVE,2.0
2013-01-03,3564.0,1.0,AUTOMOTIVE,3.0
2013-01-04,5346.0,1.0,AUTOMOTIVE,3.0
2013-01-05,7128.0,1.0,AUTOMOTIVE,5.0
...,...,...,...,...
2017-08-27,3022139.0,54.0,SEAFOOD,NaN
2017-08-28,3023921.0,54.0,SEAFOOD,NaN
2017-08-29,3025703.0,54.0,SEAFOOD,NaN
2017-08-30,3027485.0,54.0,SEAFOOD,NaN


In [50]:
df_indexes_filled.index.name = 'date'
df_indexes_filled = df_indexes_filled.reset_index()
df_indexes_filled = df_indexes_filled.sort_values(['store_nbr','family'])
df_indexes_filled = df_indexes_filled.drop_duplicates()
df_indexes_filled

,date,id,store_nbr,family,sales
0,2013-01-01,0.0,1.0,AUTOMOTIVE,0.0
1,2013-01-02,1782.0,1.0,AUTOMOTIVE,2.0
2,2013-01-03,3564.0,1.0,AUTOMOTIVE,3.0
3,2013-01-04,5346.0,1.0,AUTOMOTIVE,3.0
4,2013-01-05,7128.0,1.0,AUTOMOTIVE,5.0
...,...,...,...,...,...
3029399,2017-08-31,3029267.0,54.0,SEAFOOD,NaN
3029758,2013-12-25,NaN,54.0,SEAFOOD,NaN
3030123,2014-12-25,NaN,54.0,SEAFOOD,NaN
3030488,2015-12-25,NaN,54.0,SEAFOOD,NaN


In [51]:
last_train_date = pd.to_datetime(df_train.date.max())

In [52]:
import gc

In [53]:
del(df_train)
del(df_test)
del(df_stores)
del(df_holidays_events)
del(df_oil)
del(df_transactions)
del(df_indexes)
del(train_merged)

gc.collect()

240

## Model Building

In [54]:
from darts.models import LightGBMModel

In [55]:
future_covariates_dict['SEAFOOD'][0]

<TimeSeries (DataArray) (date: 1704, component: 20, sample: 1)>
array([[[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.79296458],
        [0.79240197],
        [0.79615402]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.79296458],
        [0.7925303 ],
        [0.79603612]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
...
        ...,
        [0.24008051],
        [0.24278495],
        [0.25032222]],

       [[0.14285714],
        [0.05714286],
        [0.14285715],
        ...,
        [0.23415849],
        [0.24149394],
        [0.25014064]],

       [[0.        ],
        [0.07142857],
        [0.15238096],
        ...,
        [0.2495558 ],
        [0.24079117],
        [0.25054285]]])
Coordinates:
  * date       (date) datetime64[ns] 2013-01-01 2013-01-02 ... 2017-08-31
  * component  (component) object 'onpromotion' 'promotion_ma_7' ... 'oil_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr   family\nglobal_compone...
    hierarchy:          None

In [56]:
'''
The function takes two arguments as input:
model_params: model hyperparameters, tuning them can improve prediction accuracy.
val_df_size: number of days in the validation set. The parameter is needed to determine
the size of the validation sample in the stacking. For blending, set the default value to zero.
'''

def lgbm_predictions(model_params, val_df_size = 0):
    l_train_date = last_train_date - np.timedelta64(val_df_size, 'D')
    local_df_indexes = df_indexes_filled.iloc[np.where(df_indexes_filled.date > l_train_date)]
    
    submission_kaggle_list = []    
    cnt = 1
    
    for params in model_params:
        LGBM_Models_Submission = {}
        display("Training...")
            
        # Fit Model
        print(f'Start fit model {cnt}')
        for family in tqdm(family_list):        
            sales_family = family_TS_transformed_dict[family]
            # training_data: represents the number of sales in the training sample minus the sales for the val
            training_data = [ts[:1688-val_df_size] for ts in sales_family]
            # TCN_covariates: represents the future covariates associated with the target product family
            TCN_covariates = future_covariates_dict[family]
            # train_sliced: represents the number of sales associated with the target product family.
            # slice_intersect: function that you can see used simply ensures that the components span the same time interval. 
            # In the case of different time intervals an error message will appear if we try to combine them.
            train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]
            

            LGBM_Model_Submission = LightGBMModel(lags = params["lags"],
                                                  lags_future_covariates = params["lags_future_covariates"],
                                                  lags_past_covariates = params["lags_past_covariates"],
                                                  output_chunk_length=1,
                                                  random_state=2022,
                                                  gpu_use_dp= "false")


            LGBM_Model_Submission.fit(series=train_sliced, 
                                  future_covariates=TCN_covariates,
                                  # transactions_transformed: the past covariates do not need to be indexed on the target 
                                  # family because there is only one global `TimeSeries` per store.
                                  past_covariates=transactions_transformed)

            LGBM_Models_Submission[family] = LGBM_Model_Submission

        display("Predictions...")
        LGBM_Forecasts_Families_Submission = {}

        # Predict
        print(f'Start predict model {cnt}')
        for family in tqdm(family_list):
            sales_family = family_TS_transformed_dict[family]
            training_data = [ts[:1688-val_df_size] for ts in sales_family]
            LGBM_covariates = future_covariates_dict[family]
            train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]

            forecast_LGBM = LGBM_Models_Submission[family].predict(
                                                  n = 16 + val_df_size,
                                                  series=train_sliced,
                                                  future_covariates=LGBM_covariates,
                                                  past_covariates=transactions_transformed
                                                 )

            LGBM_Forecasts_Families_Submission[family] = forecast_LGBM

        # Transform Back
        print(f'Start transform Back {cnt}')
        LGBM_Forecasts_Families_back_Submission = {}

        for family in tqdm(family_list):
            LGBM_Forecasts_Families_back_Submission[family] = family_pipeline_dict[family].inverse_transform(LGBM_Forecasts_Families_Submission[family], partial=True)

        # Prepare Submission in Correct Format
        print(f'Start Prepare Submission {cnt}')
        for family in tqdm(LGBM_Forecasts_Families_back_Submission):
            for n in range(0,len(LGBM_Forecasts_Families_back_Submission[family])):
                if (family_TS_dict[family][n].univariate_values()[-21:] == 0).all():
                    LGBM_Forecasts_Families_back_Submission[family][n] = LGBM_Forecasts_Families_back_Submission[family][n].map(lambda x: x * 0)

        listofseries = []

        for store in tqdm(range(0,54)):
            for family in family_list:
                oneforecast = LGBM_Forecasts_Families_back_Submission[family][store].pd_dataframe()
                oneforecast.columns = ['y_pred']
                listofseries.append(oneforecast)

        df_forecasts = pd.concat(listofseries) 
        df_forecasts.reset_index(drop=True, inplace=True)

        # No Negative Forecasts
        print(f'Start No Negative Forecasts {cnt}')
        df_forecasts[df_forecasts < 0] = 0
        forecasts_kaggle = pd.concat([local_df_indexes['id'], df_forecasts.set_index(local_df_indexes.index)], axis=1)
        forecasts_kaggle = forecasts_kaggle.reset_index(drop=True)

        # Submission
        print(f'Start Submission {cnt}')
        submission_kaggle_list.append(forecasts_kaggle)
        cnt += 1
    
    return submission_kaggle_list, local_df_indexes

In [57]:
model_params = [
    {"lags" : 63, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]},
    {"lags" : 7, "lags_future_covariates" : (16,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]},  
    {"lags" : 31, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]},
    {"lags" : 365, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]}, 
    {"lags" : 730, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]}, 
    {"lags" : 1095, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]}
]

In [58]:
submission_kaggle_list, clipped_indexes = lgbm_predictions(model_params)

'Training...'

Start fit model 1


100%|██████████| 33/33 [04:56<00:00,  9.00s/it]


'Predictions...'

Start predict model 1


...

Start transform Back 6


100%|██████████| 33/33 [00:27<00:00,  1.19it/s]


Start Prepare Submission 6


100%|██████████| 54/54 [00:02<00:00, 24.95it/s]


Start No Negative Forecasts 6
Start Submission 6


In [59]:
submissions = submission_kaggle_list[0].copy()
submissions = submissions.rename(columns={'y_pred': 'y_pred_0'})

if len(submission_kaggle_list) > 1:
    for i in range(1, len(submission_kaggle_list)):
        y_pred = submission_kaggle_list[i]
        y_pred = y_pred.rename(columns={'y_pred': f'y_pred_{i}'})
        submissions = pd.concat([submissions, y_pred.drop(['id'], axis=1)], axis=1)

submissions['sales'] = submissions.loc[:, submissions.columns!='id'].mean(axis=1)
submissions.head()

,id,y_pred_0,y_pred_1,y_pred_2,y_pred_3,y_pred_4,y_pred_5,sales
0,3000888.0,3.466254,3.328092,3.507580,3.068898,3.731862,4.001414,3.517350
1,3002670.0,2.890935,2.603473,3.216886,3.253258,3.681850,4.065727,3.285355
2,3004452.0,3.972446,2.813861,4.004264,3.852360,3.615720,2.842600,3.516875
3,3006234.0,5.056782,3.124398,5.022578,4.502551,5.044515,4.717491,4.578052
4,3008016.0,1.801655,1.053930,1.463690,1.787588,2.311553,2.026005,1.740737


In [60]:
submission = submissions[['id', 'sales']]
submission = submission.sort_values('id')
submission.id = submission.id.astype('int32')
submission.head()

,id,sales
0,3000888,3.517350
16,3000889,0.000000
32,3000890,4.315871
48,3000891,2288.981609
64,3000892,0.031684


In [61]:
submission.to_csv('/kaggle/working/submission.csv', index=False)